# Levenshtein Distance for searchbar functionality

A situation may occur in the searchbar where the user of the website may type their text incorrectly. Due to the nature of not having a spelling check for the reader (where a red underline is under the incorrectly spelled word(s)), this may make the reader confused when searching for an item and it doesnt exist.

Regex is the natural tool to solve most of these issues, however it cannot directly solve issues of incorrect spelling (at least not easily).

The Levenshtein distance is used to create a comparison score between 2 strings. The score can be described as the minimal amount of changes including insertions, reorders and deletions used to convert the first string into the desired string (string 2).

This can be characterized by the following formula which will be explained below:

$$
Lev(a, b) = |a|, \text{if}  |b| = 0, \newline
            |b|, \text{if}  |a| = 0, \newline
            |b|, \text{if}  |a| = 0, \newline
            Lev(a[1:], b[1:]) \text{if} a[0] = b[0], \newline
            1 + \min{
                    Lev(a[1:], b) \newline
                    Lev(a, b[1:]) \newline
                    Lev(a[1:], b[1:]) \newline
                    }
$$

We note that this can be quite expensive due to the recursiveness of the algorithim however using dynamic programming techniques we can reduce the amount of recursive iterations in the program.

We note that with this in mind we have a simplified time complexity of O(n**2)

# Implementation

Although we can program this partitioned problem manually, Python already comes with a library which can decide this: FuzzyWuzzy.

In [1]:
from fuzzywuzzy import fuzz

def stringComp(x, y):
    return fuzz.token_sort_ratio(x, y)
    #sort first
    #compute levenshtein distance for each string



def testcases():
    list = [
        ['canola oil', 'oil canola'],
        ['canola oil', 'canola oil'],
        ['canola oil', 'canolaoil'],
        ['oil of canola', 'canola oil'],
        ['CanolaOIL', 'CANOLAoil'],
        ['CANOLA oil', 'canola OIL'],
        ['', '']
        ]

    for i in range(len(list)):
        print(stringComp(list[i][0], list[i][1]))

In [2]:
testcases()

100
100
95
87
100
100
100


As can be seen from the results above, after each iteration, we return a score where string 1 is x% like string y. Now a consideration is due to the size of the excel file we are using, an O(n^2) algorithim for > 3000 records may be issuous when it comes to a search bar functionality where the key task is time.

Below if a function which reads all product titles and returns the scores from a given input

In [10]:
import pandas as pd
import openpyxl
import time




def read_xl():
    df = pd.read_excel('FoldedRegions.xlsx')
    return df['Product Names'].values.tolist()

def compute_lev_on_dataset(input_str):
    
    start_time = time.time()
    
    lst = read_xl()

    for i in range(len(lst)):
        stringComp(input_str, lst[i])
        
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    return elapsed_time
    
    
    

In [11]:
compute_lev_on_dataset('Canola Oil')

1.372722864151001

Interestingly enough we note that even with reading the list and then computing the levenstein distance we have an elapsed time of ~1.4 seconds.

# Concluding remarks

This feature may be quite helpful if a given score is deemed acceptable. For example, as the regex developed cannot handle incorrect spellings (in our implementation) we can have some kind of logic, where if the returned list of items is < x where x is some natural number, we can consider using the levenstein distance.

The of course will only work if the spelling of the word is somewhat accurate. For example, if we had decided to use 'HelloWorld' and the user input was 'w05l4H3llo0' this would likley yield our accuracy score useless.